In [1]:
%matplotlib inline

import os
import time

%cd /tmp
# %cd /mnt/sdb/home/tf/tensorflow/
HOME = os.getcwd()
MODEL = "efficientdet-d2"
os.environ['PYTHONPATH'] = os.path.join(HOME, "tpu")
os.environ['MODEL'] = MODEL
os.environ['CUDA_VISIBLE_DEVICES'] = ""

/tmp


In [2]:
# download the code. At the time of this writing the last
# commit is 57621e8f3eaddd2c0b421c65c0bbd323ebcf8f2d.
%cd {HOME}
!git clone https://github.com/google/automl

/tmp
Cloning into 'automl'...
remote: Enumerating objects: 3265, done.
remote: Total 3265 (delta 0), reused 0 (delta 0), pack-reused 3265
Receiving objects: 100% (3265/3265), 7.97 MiB | 7.01 MiB/s, done.
Resolving deltas: 100% (2460/2460), done.


In [3]:
%cd {HOME}/automl/efficientdet

/tmp/automl/efficientdet


In [4]:
!wget -q https://storage.googleapis.com/cloud-tpu-checkpoints/efficientdet/coco/{MODEL}.tar.gz
!tar -C /tmp -zxf {MODEL}.tar.gz

In [7]:
import tensorflow as tf
import inference
from inference import *
import PIL
import numpy as np
from PIL import Image, ImageDraw, ImageColor
import math
import matplotlib.pyplot as plt

In [16]:
class Driver(ServingDriver):
    def __init__(self, model_name, ckpt_path,
                 batch_size=1, num_classes=None, label_id_mapping=None):
        super(Driver, self).__init__(model_name, ckpt_path, batch_size, num_classes, label_id_mapping)

    def build(self, params_override=None):
        """Build model and restore checkpoints."""
        params = copy.deepcopy(self.params)
        if params_override:
            params.update(params_override)
        if not self.sess:
            self.sess = self._build_session()
            
        with self.sess.graph.as_default():
            raw_images = tf.placeholder(tf.uint8, name='image_arrays', shape=[None, None, None, 3])
            images, scales = batch_image_preprocess(raw_images, params['image_size'], self.batch_size)
            if params['data_format'] == 'channels_first':
                images = tf.transpose(images, [0, 3, 1, 2])
            class_outputs, box_outputs = build_model(self.model_name, images, **params)
            params.update(dict(batch_size=self.batch_size,pyfun=self.disable_pyfun))
            detections = det_post_process(params, class_outputs, box_outputs, scales,
                                          self.min_score_thresh,
                                          self.max_boxes_to_draw)
            restore_ckpt(
                self.sess,
                self.ckpt_path,
                ema_decay=self.params['moving_average_decay'],
                export_ckpt=None)

        self.signitures = {
            'image_arrays': raw_images,
            'prediction': detections,
        }
        return self.signitures


tf.disable_eager_execution()
tf.reset_default_graph()
output = os.path.join('.', MODEL)
if tf.io.gfile.exists(output):
    tf.io.gfile.rmtree(output)

driver = Driver(MODEL, os.path.join('/tmp', MODEL), batch_size=1)
driver.build()
driver.export(output)

AttributeError: 'Driver' object has no attribute 'disable_pyfun'